In [2]:
from CONST import *
import music21
import glob

In [3]:
# Convert files in directory
files = glob.glob("MIDI\**\*.mid", recursive=True)

979

In [8]:
for i in range(len(files)):
    print(i, files[i])

0 MIDI\0.mid
1 MIDI\0026.mid
2 MIDI\1-10_herd_gameseq.mid
3 MIDI\1.mid
4 MIDI\10.mid
5 MIDI\100.mid
6 MIDI\101.mid
7 MIDI\102.mid
8 MIDI\103.mid
9 MIDI\104.mid
10 MIDI\105.mid
11 MIDI\106.mid
12 MIDI\107.mid
13 MIDI\108.mid
14 MIDI\109.mid
15 MIDI\11.mid
16 MIDI\110.mid
17 MIDI\111.mid
18 MIDI\112.mid
19 MIDI\113.mid
20 MIDI\114.mid
21 MIDI\115.mid
22 MIDI\116.mid
23 MIDI\117.mid
24 MIDI\118.mid
25 MIDI\119.mid
26 MIDI\12.mid
27 MIDI\120.mid
28 MIDI\121.mid
29 MIDI\122.mid
30 MIDI\123.mid
31 MIDI\124.mid
32 MIDI\125.mid
33 MIDI\126.mid
34 MIDI\127.mid
35 MIDI\128.mid
36 MIDI\129.mid
37 MIDI\13.mid
38 MIDI\130.mid
39 MIDI\131.mid
40 MIDI\132.mid
41 MIDI\133.mid
42 MIDI\134.mid
43 MIDI\135.mid
44 MIDI\136.mid
45 MIDI\137.mid
46 MIDI\138.mid
47 MIDI\139.mid
48 MIDI\14.mid
49 MIDI\140.mid
50 MIDI\141.mid
51 MIDI\142.mid
52 MIDI\143.mid
53 MIDI\144.mid
54 MIDI\145.mid
55 MIDI\146.mid
56 MIDI\147.mid
57 MIDI\148.mid
58 MIDI\149.mid
59 MIDI\15.mid
60 MIDI\150.mid
61 MIDI\151.mid
62 MIDI\152.m

In [4]:
def analyzeFile(i):
    f = files[i] 

    # first, transpose the entire score into C major or A minor
    score = music21.converter.parse(f)
    key = score.analyze('key')
    mode = key.mode

    if mode == "major":
        halfSteps = majors[key.tonic.name]

    elif mode == "minor":
        halfSteps = minors[key.tonic.name]

    newscore = score.transpose(halfSteps)

    ### Now, print out score in our text format ###

    # Major/Minor
    print(mode)

    # Print parts
    for part in score.parts:
        # Find most common note
        l = [e.pitch for e in part.notes if e.isNote]
        c = [e.pitches[0] for e in part.notes if e.isChord]
        L = l+c

        # somehow have no elements in this part...
        if len(L) == 0:
            return

        m = max(set(L), key=L.count)

        print(START_CODE)

        # case on the most common pitch
        if m < music21.pitch.Pitch(MIDDLENOTE):
            print(BASS_MARK)
        else:
            print(SOPRANO_MARK)

        # prints out the notes
        for e in part.notesAndRests:
            d = float(e.duration.quarterLength)
            
            # identify the closest note length according to our defined durations
            d = NOTE_DURATIONS.index(min(NOTE_DURATIONS, key=lambda x:abs(x-d)))
            
            # CHORD. Encoded in base 12.
            if e.isChord:
                tempLen = min(3, len(e.pitches))
                
                n = 0
                
                notes = []
                
                for i in range(0, tempLen):
                    currNote = e.pitches[i]
                    
                    notes.append(int(currNote.ps))
                    
                # sort from lowest to highest
                notes.sort()
                
                for i in range(0, tempLen):
                    n += (notes[i] % OCTAVE_RANGE) * pow(OCTAVE_RANGE, i)
            
                # we MUST offset in order to account for notes and rests
                n += CHORD_OFFSET
                
            # NOTE. Encoded from 0 (c0) to 95 (b7)
            elif e.isNote:
                n = int(e.pitch.ps) - LOWEST_NOTE_OFFSET
                
                if n < 0:
                    n = n % NOTE_RANGE
                elif n >= NOTE_RANGE:
                    n = n % OCTAVE_RANGE + HIGHEST_C
                
            # REST. Just a special rest marker
            else:
                n = REST_NUM
                
            print("{0} {1}".format(n, d))
                
        print(END_CODE)

In [9]:
analyzeFile(947)

major


In [ ]:
# first, transpose the entire score into C major or A minor
score = music21.converter.parse(f)
key = score.analyze('key')
mode = key.mode

if mode == "major":
    halfSteps = majors[key.tonic.name]

elif mode == "minor":
    halfSteps = minors[key.tonic.name]

newscore = score.transpose(halfSteps)

### Now, print out score in our text format ###

# Major/Minor
print(mode)